# Предобработка данных для ВКР на тему: Исследование источников ошибок в сервисе когортного отбора на основе большой мультимодальной языковой модели  

## **Задача:** из датасета, содержащего неструктурированную информацию, выбрать данные о пациенте, диагнозе, плане лечения и ключевых лекарственных препаратах, применявшихся при его лечении. Итоговые данные должны быть в виде датасета, удобные для дальнейшей обработки. Для парсинга данных использована модель gpt-3.5-turbo
## **Датасет:** https://www.kaggle.com/datasets/tboyle10/medicaltranscriptions


In [ ]:
!pip install langchain openai tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 10.6 MB/s eta 0:00:00


In [ ]:
!pip install openai langchain huggingface_hub transformers -q

In [ ]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import openai
from openai import OpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
openai_api_key = getpass(prompt="Введите ваш OpenAI API ключ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Введите ваш OpenAI API ключ··········


In [ ]:
openai.api_key = openai_api_key

client = OpenAI(
    api_key=openai_api_key,
)

In [ ]:
llm = ChatOpenAI(temperature=0.0, openai_api_key=openai_api_key) #инициализируем языковую модель. temperature=0.0 является детерминированной, всегда выбирается ответ с наибольшей вероятностью

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/диплом/mtsamples.csv")
df.head(5)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         4999 non-null   int64 
 1   description        4999 non-null   object
 2   medical_specialty  4999 non-null   object
 3   sample_name        4999 non-null   object
 4   transcription      4966 non-null   object
 5   keywords           3931 non-null   object
dtypes: int64(1), object(5)
memory usage: 234.5+ KB


Датасет представляет из себя набор медицинских записей о пациентах. Содержит 5 столбцов с данными, 4999 строк. Описание карты пациента находется в столбце "transcription". Можно заметить, что описание не структурировано, нет единого плана опроса пациента, поле содержит много лишней информации.

In [ ]:
patient_df= df.iloc[:100, 4] # для проведения эксперимента будем использовать первые 100 строк из датасета из столбца transcription
patient_df

0     SUBJECTIVE:,  This 23-year-old white female pr...
1     PAST MEDICAL HISTORY:, He has difficulty climb...
2     HISTORY OF PRESENT ILLNESS: , I have seen ABC ...
3     2-D M-MODE: , ,1.  Left atrial enlargement wit...
4     1.  The left ventricular cavity size and wall ...
                            ...                        
95    PREOPERATIVE DIAGNOSIS: , Inguinal hernia.,POS...
96    PROCEDURE PERFORMED: , Inguinal herniorrhaphy....
97                                                  NaN
98    PREOPERATIVE DIAGNOSIS:,  Bilateral inguinal h...
99    PREOPERATIVE DIAGNOSIS: , Right inguinal herni...
Name: transcription, Length: 100, dtype: object

In [ ]:
ex = patient_df[0]
ex

'SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  TMs were clear.,Neck:  Supple without adenopathy.,

## Lang Chain
LangChain – это платформа с открытым исходным кодом для создания приложений на основе больших языковых моделей (LLM). Модели LLM обучаются по большим объемам данных и используются для глубокого обучения, чтобы генерировать ответы на запросы пользователей, например отвечать на вопросы или создавать изображения на основе текстовых подсказок. LangChain предоставляет инструменты и абстракции для расширения возможностей настройки, повышения точности и актуальности генерируемой информации.  



## Парсинг с помощью промпт инжиниринга

In [ ]:
# медицинская карта пациента
medical_card = """
SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.
She used to have allergies when she lived in Seattle but she thinks they are worse here.
In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.
She has used Allegra also.  She used that last summer and she began using it again two weeks ago.
It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.
She does have asthma but doest not require daily medication for this and does not think it is flaring up.,
MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: ,
She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,
HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.
Only clear drainage was seen.  TMs were clear.,Neck:  Supple without adenopathy.,Lungs:  Clear.,ASSESSMENT:,  Allergic rhinitis.,PLAN:,1.
She will try Zyrtec instead of Allegra again.  Another option will be to use loratadine.  She does not think she has prescription coverage so that might be cheaper.,2.
Samples of Nasonex two sprays in each nostril given for three weeks.  A prescription was written as wel
"""

# шаблон инструкции для модели
instructions_template = """
For the following text, extract the following information and provide a conclusion based on the information gathered.
Сonclusions should be concise and informative. If there is no information, put "None":

PATIENT DESCRIPTION (each point no more than 10 words):
    - Age and gender.
    - Patient weight, blood pressure.
    - Does the patient smoke, if so, for how long and how many cigarettes per day? Оther bad habits
    - Main complaints and reported symptoms.
    - Were there any operations and when?

MEDICAL HISTORY (each point no more than 15 words):
    - Medications and treatments the patient has previously tried.
    - Effectiveness and changes in response to these treatments.

DIAGNOSIS AND PLAN (each point no more than 15 words):
    - Current diagnosis based on the evaluation.
    - Proposed treatment plan, including medications and other interventions.

KEYWORDS:
    - Identify and list keywords summarizing the patient's issues and treatments.

Format the output as JSON using the following keys:
PATIENT DESCRIPTION
MEDICAL HISTORY
DIAGNOSIS AND PLAN
KEYWORDS

text: {text}
"""

In [ ]:
prompt_template = ChatPromptTemplate.from_template(instructions_template)       #класс, позволяющий создавать шаблоны промптов для использования LLM в режиме чата
prompt_template

ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='\nFor the following text, extract the following information and provide a conclusion based on the information gathered.\nСonclusions should be concise and informative. If there is no information, put "None":\n\nInclusion criteria:\n- Age 16 to 59 years\n- Infectious inflammatory diseases of throat and ears (tonsillitis, pharyngitis, otitis, etc.) recurring more than 3 times per year\n- No history of allergic reactions\nExclusion criteria:\n- Past surgical interventions on ENT organs\n- Presence of other infectious diseases (tuberculosis, hepatitis, HIV, etc.\n.\n\nFormat the output as JSON using the following keys:\nInclusion criteria\nExclusion criteria\n\ntext: {text}\n'))])

In [ ]:
messages = prompt_template.format_messages(text=medical_card)

response = llm(messages) # отправляем запрос в модель
print(response.content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{
  "Inclusion criteria": {
    "Age": "23 years",
    "Infectious inflammatory diseases": "None mentioned",
    "History of allergic reactions": "None"
  },
  "Exclusion criteria": {
    "Past surgical interventions on ENT organs": "None mentioned",
    "Presence of other infectious diseases": "None mentioned"
  }
}

Conclusion: 
Based on the information provided, the patient meets the age criteria for the study but does not have infectious inflammatory diseases of the throat and ears as required. Additionally, the patient has a history of allergic reactions, which is an exclusion criterion. Therefore, the patient would not qualify for the study.


In [ ]:
type(response.content)

str

Модель прекрасно справилась с заданием и категоризовала информацию из карты пациента. Правда данные все еще не в очень удобном для обработки информации формате.   
  
    
## Парсинг с помощью StructuredOutputParser от langchain.output_parsers
  
Для структурирования данных воспользуемся ResponseSchema и StructuredOutputParser от langchain.output_parsers.  

Langchain **StructuredOutputParser** - мощный инструмент для работы с языковыми моделями искусственного интеллекта. Позволяя определять структуру выходных данных модели, он обеспечивает гораздо более точный контроль над типом информации, генерируемой моделью. Это особенно ценно для приложений, где важно структурировать выходные данные модели определенным образом.





In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

Создадим схему ответа для модели, пропишем инструкцию на те поля, которые будут важны для нас:

In [ ]:
patient_id = ResponseSchema(name="patient",
                             description="Patient identification: indicate gender and age, if available. The patient’s profession, presence of bad habitse. If there is no data, put None ")

medical_history = ResponseSchema(name="medical_history",
                                 description="Briefly describe the patient's medical history. What diseases did you have before? What medications and treatments the patient has previously tried. Use no more than 15 words")

effect =  ResponseSchema(name="effect",
                         description="Was the previous treatment effective? Answer yes or no")

diagnos = ResponseSchema(name="diagnosis",
                         description="What diagnosis was given to him based on the patient's interview?")

plan = ResponseSchema(name="plan",
                      description="Briefly describe the proposed treatment plan, including medications and other interventions")

keywords = ResponseSchema(name="key",
                          description="Write keywords using the names of the drugs that were used to treat the patient")

In [ ]:
dict_list=[patient_id, medical_history, effect, diagnos, plan, keywords] # список полей для структурирования ответа модели

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(dict_list)

In [ ]:
format_instructions = output_parser.get_format_instructions()      # получаем инструкции по форматированию ответа
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"patient": string  // Patient identification: indicate gender and age, if available. The patient’s profession, presence of bad habitse. If there is no data, put None 
	"medical_history": string  // Briefly describe the patient's medical history. What diseases did you have before? What medications and treatments the patient has previously tried. Use no more than 15 words
	"effect": string  // Was the previous treatment effective? Answer yes or no
	"diagnosis": string  // What diagnosis was given to him based on the patient's interview?
	"plan": string  // Briefly describe the proposed treatment plan, including medications and other interventions
	"key": string  // Write keywords using the names of the drugs that were used to treat the patient
}
```


In [ ]:
# изменим шаблон запроса модели с учетом format_instructions
instructions_template_2 = """

For the following text, extract the following information and provide a conclusion based on the information gathered.
Be careful and extract all useful information from the text. If there is no information, put "None":

text: {text}

{format_instructions}

"""

In [ ]:
prompt2 = ChatPromptTemplate.from_template(template=instructions_template_2)
messages2 = prompt2.format_messages(text=medical_card, format_instructions=format_instructions)
print(messages2[0].content) # промпт для модели с учетом format_instructions



For the following text, extract the following information and provide a conclusion based on the information gathered.
Be careful and extract all useful information from the text. If there is no information, put "None":

text: 
SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.
She used to have allergies when she lived in Seattle but she thinks they are worse here.
In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.
She has used Allegra also.  She used that last summer and she began using it again two weeks ago.
It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.
She does have asthma but doest not require daily medication for this and does not think it is flaring up.,
MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: ,
She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 

In [ ]:
response2 = llm(messages2) # передаем в модель
print(response2.content)

```json
{
	"patient": "23-year-old white female",
	"medical_history": "Allergies, asthma without daily medication",
	"effect": "No",
	"diagnosis": "Allergic rhinitis",
	"plan": "Switch to Zyrtec or loratadine, Nasonex nasal spray for three weeks",
	"key": "Claritin, Zyrtec, Allegra, Ortho Tri-Cyclen, Nasonex"
}
```


In [ ]:
type(response2.content)

str

Для того,чтобы привести данные к нужному нам формату используем output_parser:

In [ ]:
output_dict = output_parser.parse(response2.content)
output_dict

{'patient': '23-year-old white female',
 'medical_history': 'Allergies, asthma without daily medication',
 'effect': 'No',
 'diagnosis': 'Allergic rhinitis',
 'plan': 'Switch to Zyrtec or loratadine, Nasonex nasal spray for three weeks',
 'key': 'Claritin, Zyrtec, Allegra, Ortho Tri-Cyclen, Nasonex'}

In [ ]:
type(output_dict)

dict

Данные теперь в формате словаря. Pаспарсим данные из исходного датасета. Для этого:
1. выполним итерации по всем медицинским катам пациентов из выборки в датасете patient_df.

2. формируеться запрос для каждой карты пациента: prompt.format_messages(). Эта функция берет текст и форматирует его в соответствии с format_instructions.

3. после формирования запроса он отправляется в Модель (llm), которая генерирует ответ.

4. обработка ответа модели и сохранение результатов output_parser.parse(response.content) включает преобразование ответа модели в структурированный формат (словарь), используя ранее созданный StructuredOutputParser.

В результате выполнения этого цикла,  получается список словарей (dict_list), где каждый словарь содержит структурированную информацию об итории болезни одного пациента из изначального датасета (patient_df).

In [ ]:
request = 1  # количество попыток обращения к API
errors = []  # ошибки

for text_input in patient_df:
    messages = prompt2.format_messages(text=text_input, format_instructions=format_instructions)
    for _ in range(request):
        try:
            response = llm(messages) # передаем запрос в модель
            break
        except:
            response = None
    if response:
        try:
            dict_list.append(output_parser.parse(response.content)) # парсим содержимое ответа модели в словарь и добавляем в список
        except:
            errors.append((len(dict_list), response)) # добавляем кортеж с индексом и некорректным ответом
            dict_list.append(None) # добавляем в список None, если не удалось обработать полученный ответ
    else:
        dict_list.append(None) # добавляем в список None, если не удалось получить ответ от API

In [ ]:
type(dict_list)

list

Теперь данные как раз в том формате, что нам нужно. Посмотрим на него

In [ ]:
dict_list

[ResponseSchema(name='patient', description='Patient identification: indicate gender and age, if available. The patient’s profession, presence of bad habitse. If there is no data, put None ', type='string'),
 ResponseSchema(name='medical_history', description="Briefly describe the patient's medical history. What diseases did you have before? What medications and treatments the patient has previously tried. Use no more than 15 words", type='string'),
 ResponseSchema(name='effect', description='Was the previous treatment effective? Answer yes or no', type='string'),
 ResponseSchema(name='diagnosis', description="What diagnosis was given to him based on the patient's interview?", type='string'),
 ResponseSchema(name='plan', description='Briefly describe the proposed treatment plan, including medications and other interventions', type='string'),
 ResponseSchema(name='key', description='Write keywords using the names of the drugs that were used to treat the patient', type='string'),
 {'pati

In [ ]:
len(dict_list)  # длина получившегося списка. Посчитаем кол-во карт пациента, которые модель не смогла обработать:

106

In [ ]:
count_of_none = dict_list.count(None)
count_of_none

1

Первые 6 строк в списке - это format_messages(). Как мы видим, модель обработала все карты пациентов кроме одной. Это замечательный результат. Удалим None в списке и сформируем датафрейм

In [ ]:
data = [x for x in dict_list if x is not None] # удалим
data = data[6:]

In [ ]:
df_rez = pd.DataFrame(data)
df_rez.head(30)

,patient,medical_history,effect,diagnosis,plan,key
0,23-year-old white female,"Allergies in Seattle, asthma without daily med...",No,Allergic rhinitis,"Switch to Zyrtec or loratadine, use Nasonex na...","Claritin, Zyrtec, Allegra, Ortho Tri-Cyclen, N..."
1,"Male, age not specified","Difficulty climbing stairs, joint pains, GERD,...",None,"Obesity, joint pain, GERD","Weight loss program, address joint pain, manag...",None
2,"Male, 42 years old, 344 pounds, 5'9""","Obesity, high cholesterol, high blood pressure...",No,Severe obesity with comorbidities,"Laparoscopic Roux-en-Y gastric bypass workup, ...","Diovan, Crestor, Tricor, Chantix"
3,None,None,None,"Left atrial enlargement, mild mitral and tricu...","Monitor left atrial enlargement, consider trea...",None
4,None,None,None,Hyperdynamic left ventricular systolic functio...,"Monitor left ventricular function, consider tr...",None
5,30-year-old female,"Morbid obesity, unsuccessful with diets",No,Morbid obesity,Laparoscopic antecolic antegastric Roux-en-Y g...,"Marcaine, VersaStep port, GIA stapler, LigaSur..."
6,31-year-old white female,Previous latissimus dorsi flap and implant bre...,No,"Deformity, right breast reconstruction; Excess...","Revision of right breast reconstruction, excis...","lidocaine, epinephrine, PDS suture, Monocryl s..."
7,None,None,None,"Normal heart and great vessel relationships, n...","No treatment required, regular monitoring reco...",None
8,None,Lipodystrophy of the abdomen and thighs,None,Lipodystrophy of the abdomen and thighs,Suction-assisted lipectomy with general anesth...,"Suction-assisted lipectomy, general anesthesia"
9,None,None,None,Normal cardiac function with trace mitral and ...,"No specific treatment required, regular monito...",None


В результате эксперимента модель прекрасно справилась с задачей и обработала 98% карт пациентов.